In [ ]:
import pandas as pd
import numpy as np
import mediapipe as mp
import cv2
import matplotlib.pyplot as plt

# Paths to the CSV files
train_path = './train.csv'
test_path = './test.csv'
# lookid_path = './IdLookupTable.csv'

# Read the CSV files
train_data = pd.read_csv(train_path)
test_data = pd.read_csv(test_path)
# lookid_data = pd.read_csv(lookid_path)

# Columns for Y_train
y_columns = [
    'left_eye_center_x','left_eye_center_y',
    'right_eye_center_x','right_eye_center_y',
    'nose_tip_x', 'nose_tip_y',
    'mouth_left_corner_x', 'mouth_left_corner_y',
    'mouth_right_corner_x', 'mouth_right_corner_y',
    'mouth_center_top_lip_x', 'mouth_center_top_lip_y',
    'mouth_center_bottom_lip_x', 'mouth_center_bottom_lip_y'
]

# Select the Image column for X_train
X_train = train_data['Image']

# Select the specified columns for Y_train
Y_train = train_data[y_columns]

# Concatenate X_train and Y_train to process all rows
combined_data = pd.concat([X_train, Y_train], axis=1)

# Mediapipe setup
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(static_image_mode=True, max_num_faces=1, min_detection_confidence=0.5)

# Function to detect face landmarks
def detect_face_landmarks(image):
    results = face_mesh.process(image)
    if results.multi_face_landmarks:
        landmarks = results.multi_face_landmarks[0].landmark
        return landmarks
    return None

# Process all images using Mediapipe
for idx, row in combined_data.iterrows():
    image = np.fromstring(row['Image'], sep=' ').reshape(96, 96)
    image_rgb = cv2.cvtColor(image.astype('uint8'), cv2.COLOR_GRAY2RGB)

    landmarks = detect_face_landmarks(image_rgb)
    if landmarks:
        # Fill the keypoints using the detected landmarks
        row['nose_tip_x'] = landmarks[4].x * 96
        row['nose_tip_y'] = landmarks[4].y * 96
        row['mouth_left_corner_x'] = landmarks[61].x * 96
        row['mouth_left_corner_y'] = landmarks[61].y * 96
        row['mouth_right_corner_x'] = landmarks[291].x * 96
        row['mouth_right_corner_y'] = landmarks[291].y * 96
        row['mouth_center_top_lip_x'] = landmarks[0].x * 96
        row['mouth_center_top_lip_y'] = landmarks[0].y * 96
        row['mouth_center_bottom_lip_x'] = landmarks[17].x * 96
        row['mouth_center_bottom_lip_y'] = landmarks[17].y * 96
        row['left_eye_center_x'] = landmarks[263].x * 96
        row['left_eye_center_y'] = landmarks[263].y * 96
        row['right_eye_center_x'] = landmarks[33].x * 96
        row['right_eye_center_y'] = landmarks[33].y * 96

        # Update the row in the combined_data DataFrame
        combined_data.loc[idx] = row

# Drop rows still having any missing values (if any)
combined_data = combined_data.dropna()

# Split the combined data back into X_train and Y_train
X_train = combined_data['Image']
Y_train = combined_data[y_columns]

# Convert X_train to numpy array and reshape
X_train = np.array([np.fromstring(image, sep=' ').reshape(96, 96, 1) for image in X_train])
X_train = X_train.astype('float32') / 255.0  # Normalize pixel values

# Convert Y_train to numpy array
Y_train = np.array(Y_train)

print(len(Y_train))



In [ ]:

# Define the model
model = Sequential()
model.add(Conv2D(32, (5, 5), input_shape=(96, 96, 1), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.1))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))
model.add(Conv2D(30, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.3))
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(256, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(14))


from keras.optimizers import Adam
#default learning rate is 0.0001. Loss curve isnt converging so lets try with higher learning rate
optimizer = Adam(learning_rate=0.001)

model.compile(optimizer=optimizer,
              loss='mean_squared_error',
              metrics=['mae', 'accuracy'])


# Compile the model
# model.compile(optimizer='adam', loss='mse')



In [ ]:
# Train the model
history = model.fit(X_train, Y_train, epochs=300, batch_size=32, validation_split=0.2)


In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

# Define a function to preprocess the image
def preprocess_image(image_path):
    # Read the image in grayscale
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    # Resize the image to 96x96
    image = cv2.resize(image, (96, 96))
    # Normalize the image
    image = image.astype('float32') / 255.0
    # Reshape the image to match the input shape of the model
    image = np.reshape(image, (1, 96, 96, 1))
    return image

# Define a function to plot the image with keypoints
def plot_image_with_keypoints(image_path, keypoints):
    # Read the image in grayscale
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    # Resize the image to 96x96
    image = cv2.resize(image, (96, 96))

    plt.imshow(image, cmap='gray')
    keypoints = keypoints.reshape(-1, 2)
    for (x, y) in keypoints:
        plt.plot(x, y, 'ro')
    plt.show()

# Preprocess the image
image_path = 'Untitled.png'
preprocessed_image = preprocess_image(image_path)

# Predict the keypoints
predicted_points = model.predict(preprocessed_image)

# Plot the image with the predicted keypoints
plot_image_with_keypoints(image_path, predicted_points[0])


In [ ]:
mode_name = '/model'
# Save the model in TensorFlow SavedModel format
model.save(mode_name)
from google.colab import files
import shutil

# Compress the saved model directory
shutil.make_archive(mode_name, 'zip', mode_name)

# Download the zipped model
files.download(mode_name+'.zip')


# Convert the model to TensorFlow Lite format
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Save the converted model to a .tflite file
tflite_model_path = 'model.tflite'
with open(tflite_model_path, 'wb') as f:
    f.write(tflite_model)

files.download(tflite_model_path)


In [ ]:
import pandas as pd
import numpy as np
import mediapipe as mp
import cv2
import matplotlib.pyplot as plt
from keras.optimizers import Adam
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense

# Paths to the CSV files
train_path = './train.csv'
test_path = './test.csv'
# lookid_path = './IdLookupTable.csv'

# Read the CSV files
train_data = pd.read_csv(train_path)
test_data = pd.read_csv(test_path)
# lookid_data = pd.read_csv(lookid_path)

# Columns for Y_train
y_columns = [
    'left_eye_center_x','left_eye_center_y',
    'right_eye_center_x','right_eye_center_y',
    'nose_tip_x', 'nose_tip_y',
    'mouth_left_corner_x', 'mouth_left_corner_y',
    'mouth_right_corner_x', 'mouth_right_corner_y',
    'mouth_center_top_lip_x', 'mouth_center_top_lip_y',
    'mouth_center_bottom_lip_x', 'mouth_center_bottom_lip_y'
]

# Select the Image column for X_train
X_train = train_data['Image']

# Select the specified columns for Y_train
Y_train = train_data[y_columns]

# Concatenate X_train and Y_train to process all rows
combined_data = pd.concat([X_train, Y_train], axis=1)

# Mediapipe setup
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(static_image_mode=True, max_num_faces=1, min_detection_confidence=0.5)

# Function to detect face landmarks
def detect_face_landmarks(image):
    results = face_mesh.process(image)
    if results.multi_face_landmarks:
        landmarks = results.multi_face_landmarks[0].landmark
        return landmarks
    return None

# Process all images using Mediapipe
for idx, row in combined_data.iterrows():
    image = np.fromstring(row['Image'], sep=' ').reshape(96, 96)
    image_rgb = cv2.cvtColor(image.astype('uint8'), cv2.COLOR_GRAY2RGB)

    landmarks = detect_face_landmarks(image_rgb)
    if landmarks:
        # Fill the keypoints using the detected landmarks
        row['nose_tip_x'] = landmarks[4].x * 96
        row['nose_tip_y'] = landmarks[4].y * 96
        row['mouth_left_corner_x'] = landmarks[61].x * 96
        row['mouth_left_corner_y'] = landmarks[61].y * 96
        row['mouth_right_corner_x'] = landmarks[291].x * 96
        row['mouth_right_corner_y'] = landmarks[291].y * 96
        row['mouth_center_top_lip_x'] = landmarks[0].x * 96
        row['mouth_center_top_lip_y'] = landmarks[0].y * 96
        row['mouth_center_bottom_lip_x'] = landmarks[17].x * 96
        row['mouth_center_bottom_lip_y'] = landmarks[17].y * 96
        row['left_eye_center_x'] = landmarks[263].x * 96
        row['left_eye_center_y'] = landmarks[263].y * 96
        row['right_eye_center_x'] = landmarks[33].x * 96
        row['right_eye_center_y'] = landmarks[33].y * 96

        # Update the row in the combined_data DataFrame
        combined_data.loc[idx] = row

# Drop rows still having any missing values (if any)
combined_data = combined_data.dropna()

# Split the combined data back into X_train and Y_train
X_train = combined_data['Image']
Y_train = combined_data[y_columns]

# Convert X_train to numpy array and reshape
X_train = np.array([np.fromstring(image, sep=' ').reshape(96, 96, 1) for image in X_train])
X_train = X_train.astype('float32') / 255.0  # Normalize pixel values

# Convert Y_train to numpy array
Y_train = np.array(Y_train)

print(len(Y_train))


# Define the model
model = Sequential()
model.add(Conv2D(32, (5, 5), input_shape=(96, 96, 1), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.1))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))
model.add(Conv2D(30, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.3))
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(256, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(14))


#default learning rate is 0.0001. Loss curve isnt converging so lets try with higher learning rate
optimizer = Adam(learning_rate=0.001)

model.compile(optimizer=optimizer,
              loss='mean_squared_error',
              metrics=['mae', 'accuracy'])


# Train the model
history = model.fit(X_train, Y_train, epochs=300, batch_size=32, validation_split=0.2)

import cv2
import numpy as np
import matplotlib.pyplot as plt

# Define a function to preprocess the image
def preprocess_image(image_path):
    # Read the image in grayscale
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    # Resize the image to 96x96
    image = cv2.resize(image, (96, 96))
    # Normalize the image
    image = image.astype('float32') / 255.0
    # Reshape the image to match the input shape of the model
    image = np.reshape(image, (1, 96, 96, 1))
    return image

# Define a function to plot the image with keypoints
def plot_image_with_keypoints(image_path, keypoints):
    # Read the image in grayscale
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    # Resize the image to 96x96
    image = cv2.resize(image, (96, 96))

    plt.imshow(image, cmap='gray')
    keypoints = keypoints.reshape(-1, 2)
    for (x, y) in keypoints:
        plt.plot(x, y, 'ro')
    plt.show()

# Preprocess the image
image_path = 'Untitled.png'
preprocessed_image = preprocess_image(image_path)

# Predict the keypoints
predicted_points = model.predict(preprocessed_image)

# Plot the image with the predicted keypoints
plot_image_with_keypoints(image_path, predicted_points[0])

